Reproject the TOPOWX meteorological data to Daymet grid, using nearest neighbor function

In [2]:
import os

os.environ['PROJDIR'] = '/gpfs/alpine/cli146/proj-shared'

import rasterio as rio
from utils.paths import *
from glob import glob
import xarray as xr
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from utils.plotting import *
from utils.analysis import *
from utils.constants import *
from osgeo import gdal

%matplotlib inline
%reload_ext autoreload
%autoreload 2

1. Check the tmin & tmax order

In [ ]:
filename = os.path.join(path_data, 'Meteorological', 'TOPOWx',
                        'fid0_19800101-19891231.tif')

t0, t1 = filename.split('_')[-1].split('-')
t0 = datetime.strptime(t0, '%Y%m%d')
t1 = datetime.strptime(t1.replace('.tif',''), '%Y%m%d')
tvec = [t0 + timedelta(days = i) for i in range((t1-t0).days+1)]
print(tvec[0], tvec[-1], len(tvec))

In [ ]:
f = rio.open(filename)
values = f.read()
values[values <= -1e+38] = np.nan
f.close()

In [ ]:
cf = plt.imshow(np.nanmean(values[len(tvec):, :, :], axis = 0))
plt.colorbar(cf)

In [ ]:
plt.plot(tvec,  np.nanmean(np.nanmean(values[:len(tvec), :, :] * 100, axis = 2), axis = 1))

In [ ]:
plt.plot(tvec,  np.nanmean(np.nanmean(values[len(tvec):, :, :] * 100, axis = 2), axis = 1))

2. Compare with the netcdf file

In [ ]:
hr = xr.open_dataset(os.path.join(path_data, 'Meteorological',
                                  'TOPOWx', 'temp',
                                  'tmax_fid0_19800101-20001231.nc'))
plt.plot(hr['time'].to_index(), hr['tmax'].mean(dim = ['lat','lon']))
hr.close()

In [ ]:
hr = xr.open_dataset(os.path.join(path_data, 'Meteorological',
                                  'TOPOWx', 'temp',
                                  'tmin_fid3_19800101-20001231.nc'))
plt.plot(hr['time'].to_index(), hr['tmin'].mean(dim = ['lat','lon']))
hr.close()

3. Reproject and save to file

In [ ]:
def reproj_topowx(fid):
    f = gdal.Open(os.path.join(path_intrim, 'urban_mask', 
                               'city_boundary_3x_merged.tif'), # 'US_urbanCluster_buffer.tif'
                               gdal.GA_ReadOnly)
    dst_srs = f.GetProjection()
    f = None

    flist = sorted(glob(os.path.join(path_data, 'Meteorological', 'TOPOWx', 'tiff_3x',
                                     f'fid{fid}_*.tif')))
    for filename in flist:
        f = gdal.Open(filename, gdal.GA_ReadOnly)
        src_srs = f.GetProjection()
        f = None

        f = gdal.Open(os.path.join(path_intrim, 'gee_single', 'NLCD', 'tiff_3x',
                                   f'NLCD_{fid:02d}.tif'), gdal.GA_ReadOnly)
        geoTransform = f.GetGeoTransform()
        minx = geoTransform[0]
        maxy = geoTransform[3]
        maxx = minx + geoTransform[1] * f.RasterXSize
        miny = maxy + geoTransform[5] * f.RasterYSize
        f = None

        print(minx, maxy, maxx, miny)

        # reproject
        newfile = filename.replace('fid', 'temp_reproj_fid')
        gdal.Warp(
            newfile,
            filename,
            format = 'GTiff',
            xRes = 1000, yRes = 1000, targetAlignedPixels = True,
            srcSRS = src_srs, dstSRS = dst_srs,
            resampleAlg = gdal.GRA_NearestNeighbour
        )

        # cut to the same extent
        newnewfile = filename.replace('fid', 'reproj_fid')
        f = gdal.Open(newfile)
        f = gdal.Translate(newnewfile, f, projWin = [minx, maxy, maxx, miny])
        f = None

        os.system(f'rm {newfile}')


for fid in range(85):
    reproj_topowx(fid)

-1633000.0 823000.0 -1542000.0 685000.0
-1633000.0 823000.0 -1542000.0 685000.0
-1633000.0 823000.0 -1542000.0 685000.0
-1633000.0 823000.0 -1542000.0 685000.0
-1633000.0 823000.0 -1542000.0 685000.0
-1633000.0 823000.0 -1542000.0 685000.0
-1633000.0 823000.0 -1542000.0 685000.0
-1633000.0 823000.0 -1542000.0 685000.0
-1267000.0 699000.0 -1208000.0 662000.0
-1267000.0 699000.0 -1208000.0 662000.0
-1267000.0 699000.0 -1208000.0 662000.0
-1267000.0 699000.0 -1208000.0 662000.0
-1267000.0 699000.0 -1208000.0 662000.0
-1267000.0 699000.0 -1208000.0 662000.0
-1267000.0 699000.0 -1208000.0 662000.0
-1267000.0 699000.0 -1208000.0 662000.0
-1709000.0 580000.0 -1633000.0 515000.0
-1709000.0 580000.0 -1633000.0 515000.0
-1709000.0 580000.0 -1633000.0 515000.0
-1709000.0 580000.0 -1633000.0 515000.0
-1709000.0 580000.0 -1633000.0 515000.0
-1709000.0 580000.0 -1633000.0 515000.0
-1709000.0 580000.0 -1633000.0 515000.0
-1709000.0 580000.0 -1633000.0 515000.0
-1282000.0 260000.0 -1225000.0 221000.0


4. Check the reprojected result

In [ ]:
for fid in range(5, 85, 20):
    print(fid)
    tmax, tmin = read_topowx(fid)

    tmax_daymet = read_daymet(fid, 'tmax')
    tmin_daymet = read_daymet(fid, 'tmin')
    tmax_daymet = tmax_daymet[tmax_daymet['time'].to_index().year <= 2016, :, :]
    tmin_daymet = tmin_daymet[tmin_daymet['time'].to_index().year <= 2016, :, :]

    fig, axes = plt.subplots(2, 2, figsize = (12, 12))

    ax = axes.flat[0]
    ax.plot(tmax['time'].to_index(), tmax.mean(dim = ['row','col']), label = 'tmax')
    ax.plot(tmin['time'].to_index(), tmin.mean(dim = ['row','col']), label = 'tmin')
    ax.legend()
    ax.set_title('TopoWX')

    ax = axes.flat[1]
    ax.plot(tmax_daymet['time'].to_index(), tmax_daymet.mean(dim = ['row','col']), label = 'tmax')
    ax.plot(tmin_daymet['time'].to_index(), tmin_daymet.mean(dim = ['row','col']), label = 'tmin')
    ax.legend()
    ax.set_title('Daymet')

    ax = axes.flat[2]
    cf = ax.imshow((tmax.mean(dim = 'time') + tmin.mean(dim = 'time'))/2)
    plt.colorbar(cf, ax = ax)
    ax.set_title('TopoWX')

    ax = axes.flat[3]
    cf = ax.imshow((tmax_daymet.mean(dim = 'time') + tmax_daymet.mean(dim = 'time'))/2)
    plt.colorbar(cf, ax = ax)
    ax.set_title('Daymet')

    fig.savefig(os.path.join(path_data, 'Meteorological', 'TOPOWx', 'tiff_3x', 'check',
                             str(fid) + '.png'), dpi = 600.)
    plt.close(fig)